<a href="https://colab.research.google.com/github/devreborges/projetos_analise_dados/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando bibliotecas

In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


Importando bibliotecas para aplicação

In [ ]:
from selenium import webdriver
import time
import os
import pandas as pd
import multiprocessing as mt
import random
from google.colab import files
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup




Carregando base de dados para realizar a raspagem de dados

In [ ]:
# Função para carregar e preparar o DataFrame
def load_dataframe(filepath):
    df = pd.read_excel(filepath)
    df['NUMERO'] = df['NUMERO'].str.replace('/', '')
    return df

Utilizando uma raspagem de dados com base em uma url para fornecer IPS valáidos para usar com proxy e evitar antibot.

In [ ]:
#função para pegar os gratuitos validos
# URL corrigido
url_proxys = "https://hidemyname.io/en/proxy-list/?country=ALARAMAUATBDBOBRBGBIKHCACLCNCOCRCZDOECEGEEFIFRGEDEHNHKHUINIDIRILITJPKZKRKGLVLYLTLUMYMXNLPEPHPLPTRORURWSARSSCSGSKZAESSECHTWTHTGTTTRGBUSUYVE&type=s&anon=4#list"

# Headers para a requisição
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"
}

def ips_proxy():
    # Fazer a requisição
    req = Request(url_proxys, headers=headers)
    response = urlopen(req)
    html = response.read()

    # Parser do BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Encontrar a tabela
    table = soup.find('table')

    # Inicializar a lista de dados
    dados = []

    # Verificar se a tabela foi encontrada
    if table:
        # Extrair todas as linhas da tabela
        for row in table.find_all('tr')[1:]:
            columns = row.find_all('td')
            if columns:
                data = [column.get_text(strip=True) for column in columns]
                dados.append(data)
    else:
        print("Nenhuma tabela encontrada")


    # Criar uma lista de IP:PORT
    ips = [f"{row[0]}:{row[1]}" for row in dados]

    # Criar um DataFrame do pandas com uma coluna única chamada 'IPS'
    df_ips = pd.DataFrame(ips, columns=['IPS'])


    # Criar a lista de proxies
    proxys = df_ips['IPS'].tolist()

    return proxys

print(ips_proxy())


['117.250.3.58:8080', '148.72.140.24:30127', '195.154.184.80:8080', '20.206.106.192:8123', '20.205.61.143:8123', '20.206.106.192:80', '152.67.9.179:8100', '20.210.113.32:80', '88.198.212.91:3128', '20.205.61.143:80', '89.43.31.134:3128']


Splitando parte do dataframe para testar a aplicação em paralelo.

In [ ]:
#Divindo uma amostra para teste
linhas_1 = df.loc[0:9, 'NUMERO']
linhas_2 = df.loc[10:19, 'NUMERO']
linhas_3 = df.loc[20:29, 'NUMERO']
linhas_4 = df.loc[30:39, 'NUMERO']



Carregando os vetores com os ids para busca

In [ ]:
ids_1 =[]
ids_2 =[]
ids_3 =[]
ids_4 =[]

#preenchendo os arrays
for i in linhas_1:
  ids_1.append(i)

for i in linhas_2:
  ids_2.append(i)

for i in linhas_3:
  ids_3.append(i)

for i in linhas_4:
  ids_4.append(i)

Função para raspagem das informações.

In [ ]:
#criando a função
def dowload_url(ids):
    proxy_usaveis = random.choice(ips_proxy())
    timer = random.randint(28, 35)
    # Configurações do navegador
    options = webdriver.ChromeOptions()
    options.add_argument(f"--proxy=https://{proxy_usaveis}")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument('--blink-settings=imagesEnabled=false')
    options.add_argument('----window-size=1,1')

    # Configurações para permitir o download automático no diretório especificado
    download_dir = '/content/downloads'
    prefs = {"download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "directory_upgrade": True}
    options.add_experimental_option("prefs", prefs)

    # Inicializar o driver do Chrome
    driver = webdriver.Chrome(options=options)

    # URL da página que queremos abrir
    url = f'https://patentscope.wipo.int/search/pt/detail.jsf?docId={ids}'

    try:
        # Abrir a página
        driver.get(url)

        # Aguardar um tempo segundos para garantir que a página seja totalmente carregada
        time.sleep(timer)

        # Capturar o conteúdo da página
        page_content = driver.page_source

        # Salvar o conteúdo em um arquivo
        file_path = os.path.join(download_dir, f"{ids}.html")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(page_content)

        print("Arquivo salvo em:", file_path)

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        time.sleep(2)
        print("Tentando novamente...")
        download_url(ids)
    finally:
        # Fechar o navegador
        driver.quit()



Armazenando cada função em uma variável para rodar em paralelo

In [ ]:
def part1():
  for i in ids_1:
    dowload_url(i)

def part2():
  for i in ids_2:
    dowload_url(i)

def part3():
  for i in ids_3:
    dowload_url(i)

def part4():
  for i in ids_4:
    dowload_url(i)

Aplicando o paralelismo

In [ ]:
#rodando a aplicação em paralelo.
inicio = time.perf_counter()

if __name__ == '__main__':
    # Cria um processo para executar part1
    cod1 = mt.Process(target=part1)
    cod2 = mt.Process(target=part2)
    cod3 = mt.Process(target=part3)
    cod4 = mt.Process(target=part4)

    # Inicia o processo
    cod1.start()
    cod2.start()
    cod3.start()
    cod4.start()

    # Espera pelo término do processo
    cod1.join()
    cod2.join()
    cod3.join()
    cod4.join()

    fim = time.perf_counter()
    total = round(fim - inicio, 2)
    print(f'O tempo de execução foi: {total} segundos')

Arquivo salvo em: /content/downloads/WO2022268657.html
Arquivo salvo em: /content/downloads/WO2023279092.html
Arquivo salvo em: /content/downloads/WO2023274914.html
Arquivo salvo em: /content/downloads/WO2023285075.html
Arquivo salvo em: /content/downloads/WO2022268054.html
Arquivo salvo em: /content/downloads/WO2022263418.html
Arquivo salvo em:
 /content/downloads/WO2011111756.htmlArquivo salvo em:
 /content/downloads/WO2023009753.htmlArquivo salvo em: /content/downloads/WO2011128322.html
Arquivo salvo em:
 /content/downloads/WO2023278907.htmlArquivo salvo em: /content/downloads/WO2011123632.html
Arquivo salvo em: /content/downloads/WO2023012135.html
Arquivo salvo em: /content/downloads/WO2012006264.html
Arquivo salvo em: /content/downloads/WO2023275025.html
Arquivo salvo em: /content/downloads/WO2012045646.html
Arquivo salvo em: /content/downloads/WO2023286392.html
Arquivo salvo em: /content/downloads/WO2012048814.html
Arquivo salvo em: /content/downloads/WO2022268291.html
Arquivo sa

Exportando o dataframe com a base de dados preenchida para análises

In [ ]:
def expor_dataframe(pasta_downloads="/content/downloads", output_file="mdandatarios_bolados.xlsx"):
    """
    Função para iterar por todos os arquivos HTML em uma pasta, extrair o texto da tag
    <span> com a classe 'biblio-person-list--name', e exportar os resultados para um arquivo Excel.

    Parâmetros:
    pasta_downloads (str): Caminho para a pasta onde os arquivos HTML estão localizados.
    output_file (str): Nome do arquivo Excel de saída.
    """
    # Iniciar a contagem do tempo
    inicio = time.perf_counter()

    # Lista para armazenar os dados
    data_base = []

    # Iterar por todos os arquivos na pasta downloads
    for nome_arquivo in os.listdir(pasta_downloads):
        caminho_arquivo = os.path.join(pasta_downloads, nome_arquivo)

        # Verificar se o caminho é de um arquivo
        if os.path.isfile(caminho_arquivo) and nome_arquivo.endswith('.html'):
            # Abrir o arquivo e ler o conteúdo
            with open(caminho_arquivo, "r", encoding="utf-8") as arquivo:
                texto_do_arquivo = arquivo.read()

            # Criar um objeto BeautifulSoup com o conteúdo do arquivo
            soup = BeautifulSoup(texto_do_arquivo, 'html.parser')

            # Encontrar a tag <span> com a classe 'biblio-person-list--name'
            span = soup.find('span', class_='biblio-person-list--name')

            # Se a tag span for encontrada, extrair o texto dela
            if span:
                texto_com_quebras_de_linha = span.get_text(strip=True)
            else:
                texto_com_quebras_de_linha = "Tag não encontrada"

            # Adicionar os dados na lista
            data_base.append({'ID': nome_arquivo, 'MANDATARIOS': texto_com_quebras_de_linha})

    # Criar um DataFrame com os dados
    df = pd.DataFrame(data_base)

    # Exportar o DataFrame para um arquivo Excel
    df.to_excel(output_file, index=False)

    # Calcular o tempo total de execução
    fim = time.perf_counter()
    total = round(fim - inicio, 6)

    print(f'O tempo de execução foi: {total}')

In [ ]:
expor_dataframe()

O tempo de execução foi: 9.20485


Carregando o dataframe exportando para tratamento

In [ ]:
df_mandatarios = pd.read_excel('mdandatarios_bolados.xlsx')

In [ ]:
df_mandatarios.rename(columns={'ID': 'NUMERO'}, inplace=True)


In [ ]:
df_mandatarios.head()

,NUMERO,MANDATARIOS
0,WO2022182502.html,QUALCOMM INCORPORATED
1,WO2023278907.html,QUALCOMM INCORPORATED
2,WO2023019816.html,天翼数字生活科技有限公司 E-SURFING DIGITAL LIFE TECHNOLOGY...
3,WO2022226100.html,SEAGEN INC.
4,WO2022262535.html,"华为技术有限公司 HUAWEI TECHNOLOGIES CO., LTD."


In [ ]:
df_mandatarios['NUMERO'] = df_mandatarios['NUMERO'].str.replace('.html', '')
df_mandatarios.head()

,NUMERO,MANDATARIOS
0,WO2022182502,QUALCOMM INCORPORATED
1,WO2023278907,QUALCOMM INCORPORATED
2,WO2023019816,天翼数字生活科技有限公司 E-SURFING DIGITAL LIFE TECHNOLOGY...
3,WO2022226100,SEAGEN INC.
4,WO2022262535,"华为技术有限公司 HUAWEI TECHNOLOGIES CO., LTD."


In [ ]:
df_mandatarios['NUMERO'] = df_mandatarios['NUMERO'].str[:6] + '/' + df_mandatarios['NUMERO'].str[6:]
df_mandatarios.head()

,NUMERO,MANDATARIOS
0,WO2022/182502,QUALCOMM INCORPORATED
1,WO2023/278907,QUALCOMM INCORPORATED
2,WO2023/019816,天翼数字生活科技有限公司 E-SURFING DIGITAL LIFE TECHNOLOGY...
3,WO2022/226100,SEAGEN INC.
4,WO2022/262535,"华为技术有限公司 HUAWEI TECHNOLOGIES CO., LTD."


Exportando e realizando download do daframe após a o tratamento de dados

In [ ]:
df_mandatarios.to_excel('teste_processamento_2.xlsx', index=False)



In [ ]:
files.download('teste_processamento_2.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>